# DSCI 525 - Web and Cloud Computing

## Group 14
- 	Nishar	Dhruvi	
- 	Ouedraogo Flora
- 	Hong	Eyre
- 	Kohli	Manvir

In [56]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [57]:


# change to the directory that you want to download
%cd /Users/floraouedraogo/Documents/DSCI/DSCI 525/figshareexp

/Users/floraouedraogo/Documents/DSCI/DSCI 525/figshareexp


In [58]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [59]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files


[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [60]:

%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.41 s, sys: 7.23 s, total: 10.6 s
Wall time: 1min 26s


In [61]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.8 s, sys: 1.13 s, total: 8.92 s
Wall time: 10.4 s


In [62]:

%ls -ltr figsharerainfall/

total 12428672
drwxr-xr-x@ 30 floraouedraogo  staff        960 Mar 30 00:18 __MACOSX/
-rw-r--r--   1 floraouedraogo  staff  814041183 Mar 30 15:12 data.zip
-rw-r--r--   1 floraouedraogo  staff   95376895 Mar 30 15:12 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff   94960113 Mar 30 15:12 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff   82474546 Mar 30 15:12 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  127613760 Mar 30 15:12 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  232118894 Mar 30 15:12 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  330360682 Mar 30 15:12 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  254009247 Mar 30 15:12 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  235661418 Mar 30 15:12 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 floraouedraogo  staff  294260911 Mar 30 15:12 BCC-CSM2-MR_d

Combining data CSVs

In [63]:
%%time
import pandas as pd
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("combined_data.csv")

CPU times: user 3min 21s, sys: 10.4 s, total: 3min 32s
Wall time: 3min 37s
